# Ottimizzazione degli iperparametri

La fase di ottimizzazione degli iperparametri, conosciuta in gergo tecnico come *hyperparameters tuning*, è la parte più ostica nel processo di creazione di un modello predittivo.<br>
In questo notebook vedremo due tecniche che ci permettono di semplificare questo lavoro.
<br><br>
Importiamo le librerie necessarie.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Carichiamo l'Iris Dataset all'interno di un DataFrame.

In [2]:
iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", 
                   names=["sepal length","sepal width","petal length","petal width","class"])
iris.head()

sepal length  sepal width  petal length  petal width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa

E creiamo gli array per addestramento e test.

In [3]:
X = iris.drop("class",axis=1).values
Y = iris["class"].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

## Grid Search

La prima tecnica che vediamo è la Grid Search, che permette di ricercare gli iperparametri corretti con un approccio a forza bruta.<br>
Possiamo implementare il Grid Search utilizzando la classe <span style="font-family: Monaco">GridSearchCV</span>
 di scikit-learn.<br>
Questa classe effettua la grid search su un'insieme di parametri che definiamo tramite un dizionario e valida i modelli ottenuti tramite k-fold cross-validation.

In [20]:
%%time

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svc = SVC()

param_grid = {"kernel": ["linear","rbf","sigmoid","poly"],
             "C": [1, 10, 100, 1000],
             "gamma": [0.1,1,"auto"]}

gs = GridSearchCV(svc, param_grid, cv=10) #in cv specifichiamo il numero di folds per la cross-validation

gs.fit(X_train, Y_train)

CPU times: total: 766 ms
Wall time: 757 ms


GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000], 'gamma': [0.1, 1, 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid', 'poly']})

Possiamo ottenere gli iperparametri e l'accuracy del modello migliore tramite gli attributi <span style="font-family: Monaco">best_params_</span> e <span style="font-family: Monaco">best_score_</span>.

In [21]:
print("Iperparametri del modello migliore: %s" % gs.best_params_)
print("Accuracy=%.4f" % gs.best_score_)

Iperparametri del modello migliore: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Accuracy=0.9800


Possiamo anche ottenere direttamente il modello migliore già addestrato tramite l'attributo <span style="font-family: Monaco">best_estimator</span>.<br>
Recuperiamo il modello migliore trovato durante la grid search e calcoliamo l'accuracy sul test set.

In [22]:
svc = gs.best_estimator_
svc.score(X_test,Y_test)

0.9777777777777777

## Random Search
La seconda tecnica che vediamo è la Random Search, che ricerca i valori ottimali degli iperparametri cercandoli a caso in una distribuzione di valori da noi definita.<br>
Evidenze sperimentali hanno dimostrato che questo approccio porta a risultati migliori e più velocemente rispetto alla Grid Search.<br>
Possiamo implementare la Random Search utilizzando la classe RandomizedSearchCV di scikit-learn.<br>
Questa classe effettua la random search su una distribuzione di parametri che definiamo tramite un dizionario e valida i modelli ottenuti tramite k-fold cross-validation.

In [25]:
%%time

from sklearn.model_selection import RandomizedSearchCV

svc = SVC()

param_dist = {"kernel": ["linear","rbf","sigmoid","poly"],
             "C": [1, 10, 100, 1000],
             "gamma": [0.1,1,"auto"]}

rs = RandomizedSearchCV(svc, param_dist, cv=10)

rs.fit(X_train, Y_train)

print("Iperparametri migliori: %s" % rs.best_params_)
print("Accuracy=%.4f" % rs.best_score_)

Iperparametri migliori: {'kernel': 'linear', 'gamma': 0.1, 'C': 1}
Accuracy=0.9800
CPU times: total: 172 ms
Wall time: 173 ms


La Random Search ha trovato il modello migliore in meno di un quinto del tempo della Grid Search.<br>
Recuperiamo il modello migliore trovato durante la random search e calcoliamo l'accuracy sul test set.

In [26]:
rs = rs.best_estimator_
rs.score(X_test,Y_test)

0.9777777777777777

**Nota**<br>
Rieseguendo più volte lo stesso codice alla cella 25 ottengo risultati diversi